In [1]:
# Initial Imports

import pandas as pd
import numpy as np
import requests
import json
import datetime as dt
from pathlib import Path
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Import flair pre-trained sentiment model
from flair.models import TextClassifier
classifier = TextClassifier.load('en-sentiment')
# Import flair Sentence to process input text
from flair.data import Sentence
import regex as re
from sentiment_functions import clean_text, score_flair
import datetime
# BDay is business day, not birthday...
from pandas.tseries.offsets import BDay




ModuleNotFoundError: No module named 'vaderSentiment'

In [3]:
# Taking user input for number of days to retrieve data for, then creating the date range to iterate through for the API call


def getTwitterData():
        
    tweets_df = pd.DataFrame()
    numDays = int(input('How many days would you like to retrieve twitter data for?'))
    hashtag = input('What hashtag would you like to retrieve data for?')
    
    today = datetime.datetime.today()
    start = today - pd.Timedelta(days = numDays)
    
    

    #Since we are retrieving data we want to correlate with markets, we are using 'bdate_range' to only return business days                
    date_range = pd.bdate_range(start=start, end=today, freq="D", tz='UTC') + pd.Timedelta(hours=5)
    display(date_range)
    
    #Iterating through the date_range with an API call for each day to maximize data returned 
    # @TODO -- use function to limit API calls to not exceed limits
    for i in range(0, len(date_range) -1, 1):
    
        start_date = date_range[i]
        end_date = date_range[i+1] 
    
    
        start_date = pd.Timestamp.isoformat(start_date)
        end_date = pd.Timestamp.isoformat(end_date)
    
    
# Making the API call to retrieve tweets 

        source = "https://twitter32.p.rapidapi.com/getSearch"

# These parameters need to be set before running API call... could in the future receive inputs from user
## TODO -- if input is received, datetime will need to be formatted correctly for API call

        hashtag = hashtag

        querystring = {"hashtag": hashtag, "start_date": start_date,"end_date": end_date,"lang":"en"}
        headers = {
        'x-rapidapi-host': "twitter32.p.rapidapi.com",
        'x-rapidapi-key': "49e5cedc9fmsh6a2df83dacfc4c1p1c3469jsn3edb2e8cb9df"
        }
   
        
        response = requests.get(source, headers=headers, params=querystring).json()
        df = pd.DataFrame(response['data']['tweets']).T
        tweets_df = pd.concat([df, tweets_df])
        display(start_date, end_date)
    
    return tweets_df

tweets_df = getTwitterData()
display(tweets_df)


        


How many days would you like to retrieve twitter data for? 5
What hashtag would you like to retrieve data for? $TSLA


DatetimeIndex(['2022-01-12 05:00:00+00:00', '2022-01-13 05:00:00+00:00',
               '2022-01-14 05:00:00+00:00', '2022-01-15 05:00:00+00:00',
               '2022-01-16 05:00:00+00:00', '2022-01-17 05:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

'2022-01-12T05:00:00+00:00'

'2022-01-13T05:00:00+00:00'

'2022-01-13T05:00:00+00:00'

...

'2022-01-15T05:00:00+00:00'

'2022-01-16T05:00:00+00:00'

'2022-01-16T05:00:00+00:00'

'2022-01-17T05:00:00+00:00'

,contributors,conversation_id,conversation_id_str,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,...,possibly_sensitive_editable,quote_count,reply_count,retweet_count,retweeted,source,supplemental_language,truncated,user_id,user_id_str
1482146142339178499,None,1482146142339178499,1482146142339178499,None,Sat Jan 15 00:22:31 +0000 2022,"[0, 214]","{'hashtags': [{'text': 'wallstreetbets', 'indi...","{'media': [{'id': 1482146140653072387, 'id_str...",0,False,...,True,0,0,0,False,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",None,False,1422573397079375877,1422573397079375877
1481783730788352001,None,1481783730788352001,1481783730788352001,None,Fri Jan 14 00:22:26 +0000 2022,"[0, 212]","{'hashtags': [{'text': 'wallstreetbets', 'indi...","{'media': [{'id': 1481783728095510533, 'id_str...",0,False,...,True,0,0,0,False,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",None,False,1422573397079375877,1422573397079375877
1481421406487420934,None,1481421406487420934,1481421406487420934,None,Thu Jan 13 00:22:41 +0000 2022,"[0, 210]","{'hashtags': [{'text': 'wallstreetbets', 'indi...","{'media': [{'id': 1481421404000165888, 'id_str...",1,False,...,True,0,0,2,False,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",None,False,1422573397079375877,1422573397079375877
1481277539410710533,None,1481277539410710533,1481277539410710533,None,Wed Jan 12 14:51:00 +0000 2022,"[0, 276]","{'hashtags': [], 'symbols': [{'text': 'qqq', '...",NaN,0,False,...,NaN,0,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",None,False,1286447494222745600,1286447494222745600


In [ ]:
'''def getTodayTweets(current_df_path)
    
    todaysTweets_df = pd.DataFrame()
    
    current_df_path = Path('./twitter_flair.csv'
    currentTweets_df = pd.read_csv(current_df_path, parse_dates = True, infer_datetime_format=True, index_col='Date')
    hashtag = input('Please input desired hashtag to confirm')
    
    end_date = datetime.datetime.today()
    start_date = today - pd.Timedelta(days=1)
    
    
    #Iterating through the date_range with an API call for each day to maximize data returned 
    # @TODO -- use function to limit API calls to not exceed limit
    
    start_date = pd.Timestamp.isoformat(start_date)
    end_date = pd.Timestamp.isoformat(end_date)
    
    
    # Making the API call to retrieve tweets 

    source = "https://twitter32.p.rapidapi.com/getSearch"

    # These parameters need to be set before running API call... could in the future receive inputs from user
    ## TODO -- if input is received, datetime will need to be formatted correctly for API call

    hashtag = hashtag
    querystring = {"hashtag": hashtag, "start_date": start_date,"end_date": end_date,"lang":"en"}
    headers = {
        'x-rapidapi-host': "twitter32.p.rapidapi.com",
        'x-rapidapi-key': "49e5cedc9fmsh6a2df83dacfc4c1p1c3469jsn3edb2e8cb9df"
        }
   
        
    response = requests.get(source, headers=headers, params=querystring).json()
    todaysTweets_df = pd.DataFrame(response['data']['tweets']).T
    updatedTweets_df = pd.concat([todays_Tweets_df, currentTweets])
    display(start_date, end_date)
                           
    
    return updatedTweets_df

updatedTweets_df = getTwitterData('./twitter_flair.csv')
display(tweets_df)'''

In [4]:
# Parsing and formatting the dataframe to index by date created and isolating the 'full_text' and 'retweet_count' columns for sentiment analysis
df_filtered = tweets_df.loc[:,["created_at","full_text", 'retweet_count']]
df_filtered.index = pd.to_datetime(df_filtered['created_at'], utc=True)
df_filtered.index.name = 'Date'
df_filtered.drop(columns='created_at', inplace=True)
df_filtered


,full_text,retweet_count
Date,,
2022-01-15 00:22:31+00:00,Following were the Top 5 tickers mentioned on ...,0
2022-01-14 00:22:26+00:00,Following were the Top 5 tickers mentioned on ...,0
2022-01-13 00:22:41+00:00,Following were the Top 5 tickers mentioned on ...,2
2022-01-12 14:51:00+00:00,advise needed! - I mistakenly bought an option...,0


In [5]:
# We need to clean up our tweets so they can be analyzed in string format
# Applying our imported 'clean_text' function to text column of our dataframe
df_filtered["full_text"] = df_filtered["full_text"].apply(clean_text)
df_filtered = df_filtered.sort_index()
df_filtered

,full_text,retweet_count
Date,,
2022-01-12 14:51:00+00:00,advise needed I mistakenly bought an option wi...,0
2022-01-13 00:22:41+00:00,Following were the Top 5 tickers mentioned on ...,2
2022-01-14 00:22:26+00:00,Following were the Top 5 tickers mentioned on ...,0
2022-01-15 00:22:31+00:00,Following were the Top 5 tickers mentioned on ...,0


In [6]:


# Get sentiment score for each review
df_filtered['scores_flair'] = df_filtered['full_text'].apply(lambda s: score_flair(s)[0])
# Predict sentiment label for each review
df_filtered['pred_flair'] = df_filtered['full_text'].apply(lambda s: score_flair(s)[1])
# Check the distribution of the score
df_filtered['scores_flair'].describe()
# Change the label of flair prediction to 0 if negative and 1 if positive
#mapping = {'NEGATIVE': 0, 'POSITIVE': 1}
#df_filtered['pred_flair_numerical'] = df_filtered['pred_flair'].map(mapping)
df_filtered['pred_flair'].describe()


count            4
unique           1
top       NEGATIVE
freq             4
Name: pred_flair, dtype: object

In [7]:
#Converting the categorical variable columns into numerical with get_dummies
df_dummies = pd.get_dummies(df_filtered['pred_flair'])
# Dropping the row if the scores_flair confidence level is below 0.75
df_filtered.drop(df_filtered[df_filtered.scores_flair < 0.75].index, inplace=True)
# Creating the 'df_signals' dataframe that we will use to append to our primary dataset, using only the columns with relevant numerical values, resampled by day
df_signals = df_filtered.drop(columns=['scores_flair', 'pred_flair'])
final_df = pd.concat([df_signals, df_dummies]).resample('D').sum()
display(final_df)  
#final_df.to_csv('twitter_flair.csv')

    
    


,NEGATIVE
Date,
2022-01-12 00:00:00+00:00,1.0
2022-01-13 00:00:00+00:00,1.0
2022-01-14 00:00:00+00:00,1.0
2022-01-15 00:00:00+00:00,1.0


### After pulling tweet data, performing sentiment analysis using flair and then filtered and resampling the dataframe, we return a dataframe with datetime indices with two columns -- flair_negative, which is the sum of all negative tweets for the day, and flair_positive, which is the sum of all positive tweets for the day -- at some point we could think about factoring in number of retweets or something along those lines to weigh the tweets by their impact, but the results from the API call already filter by this metric, so could be superfluous.